In [2]:
import numpy as np
import pandas as pd
import scipy
from typing import TypeVar,Mapping, Set, Generic, Sequence
from scipy.stats import norm

### Black-Scholes formulas for European Call/Put Pricing
- Underlying asset price
$$S_t = \mu S_t dt + \sigma S_t dz_t$$
- Value of call option
$$dC(S,t)= (\mu S \frac{dC}{dS} + \frac{dC}{dt} + \frac{1}{2} \sigma^2 S^2 \frac{d^2C}{dS^2})dt + \sigma S \frac{dC}{dS} dz$$
- Replicate call option with cash ($x_t$ units) and underlying asset ($y_t$ units)
$$P_t = x_t B_t + y_t S_t$$
$$dP_t = (r x_t B_t + y_t \mu S_t) dt + y_t \sigma S_t d z_t$$
- Match $dC(S,t)$ with $dP_t$
$$y_t = \frac{dC}{dS}$$
$$r x_t B_t = \frac{dC}{dt} + \frac{1}{2} \sigma^2 S^2 \frac{d^2C}{dS^2}$$
- Match $C_t$ with $P_t$
$$r C_t = \frac{dC}{dt} + \frac{1}{2} \sigma^2 S^2 \frac{d^2C}{dS^2} + r \frac{dC}{dS} S_t$$
- Solve PDE with $C(S,T) = \max \{S-K, 0\}$ and $C(0,t) = 0$.
$$C(S,t) = S_t \Phi(d_1) - e^{-r(T-t)} K \Phi(d_2)$$
$$d_1 = \frac{\log(\frac{S_t}{K}) + (r +\sigma^2/2) (T-t)}{\sqrt{T-t}}$$
$$d_2 = d_1 -\sigma \sqrt{T-t}$$

In [4]:
def european_call(S: float, K: float, T: int, r: float, sigma: float, t: int) -> float:   
    # S, spot price
    # K, strike price
    # T, maturity time
    # t, current time
    # r: risk free rate
    # sigma: sd of underlying asset
    
    t_gap = T-t
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * t_gap) / (sigma * np.sqrt(t_gap))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * t_gap) / (sigma * np.sqrt(t_gap))
    call = (S * norm.cdf(d1, 0, 1) - K * np.exp(-r * t_gap) * norm.cdf(d2, 0, 1))
    return call

def european_put(S: float, K: float, T: int, r: float, sigma: float, t: int) -> float:   
    # S, spot price
    # K, strike price
    # T, maturity time
    # t, current time
    # r: risk free rate
    # sigma: sd of underlying asset
    
    t_gap = T-t
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T-t) / (sigma * np.sqrt(T-t))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T-t) / (sigma * np.sqrt(T-t))
    put = (K * np.exp(-r * T-t) * norm.cdf(-d2, 0, 1) - S * norm.cdf(-d1, 0, 1))
    
    return put

### Standard binary tree/grid-based numerical algorithm for American Option Pricing 
Ref: https://maths.ucd.ie/~vlasenko/MST30030/fm17.pdf

In [7]:
def BinaryTree_call(S: float, K: float, T: int, r: float, sigma: float, t: int) -> float:  
    # u,d,p: https://en.wikipedia.org/wiki/Binomial_options_pricing_model
    u = np.exp(sigma*np.sqrt(t/T))
    d = np.exp(-sigma*np.sqrt(t/T))
    p = (np.exp(r*t/T)-d) / (u-d) 

    val = np.zeros((T+1,T+1))
    val[0,0] = S
    for i in range(1,T+1):
        val[i,0] = val[i-1,0]*u
        for j in range(1,i+1):
            val[i,j] = val[i-1,j-1]*d
    option_val = np.zeros_like(val)
    for j in range(T+1):
        option_val[T,j] = max(0, val[T,j]-K)
    for i in range(T-1,-1,-1): # Backward
        for j in range(i+1):
            option_val[i,j] = max(0, val[i,j]-K, np.exp(-r*(t/T))*(p*option_val[i+1,j]+(1-p)*option_val[i+1,j+1]))
    return option_val[0,0]

def BinaryTree_put(S: float, K: float, T: int, r: float, sigma: float, t: int) -> float:  
    # u,d,p: https://en.wikipedia.org/wiki/Binomial_options_pricing_model
    u = np.exp(sigma*np.sqrt(t/T))
    d = np.exp(-sigma*np.sqrt(t/T))
    p = (np.exp(r*t/T)-d) / (u-d) 

    val = np.zeros((T+1,T+1))
    val[0,0] = S
    for i in range(1,T+1):
        val[i,0] = val[i-1,0]*u
        for j in range(1,i+1):
            val[i,j] = val[i-1,j-1]*d
    option_val = np.zeros_like(val)
    for j in range(T+1):
        option_val[T,j] = max(0, K - val[T,j])
    for i in range(T-1,-1,-1):
        for j in range(i+1):
            option_val[i,j] = max(0, K - val[i,j], np.exp(-r*(t/T))*(p*option_val[i+1,j]+(1-p)*option_val[i+1,j+1]))
    return option_val[0,0]

### Longstaff-Schwartz Algorithm
Ref: http://www2.maths.ox.ac.uk/~gilesm/mc/nus/lec6.pdf

In [8]:
def ind(x):
    return x

def LongstaffSchwartz_call(SP: np.ndarray, r: float, phi=ind) -> float:
    # SP[0:m, 0:n+1], i: path, j: timestep
    # r, discount rate
    # function
    m,n = SP.shape
    CF = [max(SP[i,n]-K,0) for i in range(m)]
    for j in range(n-1,-1,0):
        CF = CF * np.exp(-r)
        X = np.asarray([phi(SP[i,j]) for i in range(m) if max(SP[i,j]-K,0)>0]).reshape(-1,1)
        Y = np.asarray([CF(i) for i in range(m) if max(SP[i,j]-K,0)>0]).reshape(-1,1)
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
        for i in range(m):
            if max(SP[i,j]-K,0) > w*phi(SP[i,j]):
                CF[i] = max(SP[i,j]-K,0)
    exercise = max(SP[0,0]-K,0)
    _continue = np.exp(-r)*np.mean(CF)
    return (max(exercise,_continue))